In [1]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    ISOT
'''



data = pd.read_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv")



In [51]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

# Scale embeddings


#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
filtered = data.dropna()
classes = filtered["flag"].to_numpy()
#features = filtered[["title_vader_class", "text_vader_class", "text_tb_pol_class", "text_tb_sub_class", "title_tb_pol_class", "title_tb_sub_class"]].dropna()

'''
  Tweet classifier
["text_NN_tweets", "title_NN_tweets", "text_log_tweets", title_log_tweets"]
'''

tweet_features = ["text_NN_tweets", "title_NN_tweets", "text_log_tweets", "title_log_tweets", "text_tb_sub_class", "title_tb_sub_class"]
imdb_features = ["text_NN_imdb", "title_NN_imdb", "text_log_imdb", "title_log_imdb"]

tweet_and_imdb = [i for i in tweet_features]
tweet_and_imdb.extend(imdb_features)

features = filtered[["text_tb_pol",	"text_tb_sub",	"title_tb_pol",	"title_tb_sub",	"title_vader_comp",	"title_vader_neg",	
              "title_vader_neu",	"title_vader_pos",	"text_vader_comp",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]].to_numpy()


#for i in range(100):
    
x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=42)
#x_train

ensemble = LogisticRegression()
ensemble.fit(x_train, y_train)

# Evaluate predictions
y_pred = ensemble.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Ensemble accuracy, textblob and vader: {accuracy * 100:.2f}%")



uniques, counts = np.unique(y_test, return_counts=True)
counts







Ensemble accuracy, textblob and vader: 71.19%


array([3510, 3221], dtype=int64)

In [50]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(12, 1)),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')
])



model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
'''text only'''
filtered = data.dropna()
classes = filtered["flag"].to_numpy()
#features = filtered[["title_vader_class", "text_vader_class", "text_tb_pol_class", "text_tb_sub_class", "title_tb_pol_class", "title_tb_sub_class"]].to_numpy()
features = filtered[["text_tb_pol",	"text_tb_sub",	"title_tb_pol",	"title_tb_sub",	"title_vader_comp",	"title_vader_neg",	
              "title_vader_neu",	"title_vader_pos",	"text_vader_comp",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]].to_numpy()

'''
  Tweet classifier
["text_NN_tweets", "title_NN_tweets", "text_log_tweets", title_log_tweets"]
'''

tweet_features = ["text_NN_tweets", "title_NN_tweets", "text_log_tweets", "title_log_tweets"]
#imdb_features = ["text_NN_imdb", "title_NN_imdb", "text_log_imdb", "title_log_imdb"]

tweet_and_imdb = [i for i in tweet_features]
tweet_and_imdb.extend(imdb_features)

#features = filtered[tweet_and_imdb].to_numpy()

x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=42)
model.fit(x_train, y_train, epochs=10, verbose=2)

c:\Users\desmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1192/1192 - 2s - 2ms/step - accuracy: 0.6475 - loss: 0.6235
Epoch 2/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7199 - loss: 0.5606
Epoch 3/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7340 - loss: 0.5410
Epoch 4/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7437 - loss: 0.5273
Epoch 5/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7500 - loss: 0.5156
Epoch 6/10
1192/1192 - 1s - 962us/step - accuracy: 0.7548 - loss: 0.5064
Epoch 7/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7597 - loss: 0.4993
Epoch 8/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7642 - loss: 0.4935
Epoch 9/10
1192/1192 - 3s - 2ms/step - accuracy: 0.7669 - loss: 0.4897
Epoch 10/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7684 - loss: 0.4859
